In [4]:
import mysql.connector
import json
import sys
from pyjarowinkler import distance
import pandas as pd
from datetime import date
import networkx as nx

In [34]:
sample_data = {
    'cleaned_name': 'EDY PRIYONO S',
    'cleaned_TEMPAT_LAHIR': 'SEMARANG',
    'TGL_LAHIR': date(1967, 10, 1),  # assumed placeholder; change if needed
    'cleaned_no_ktp': None,
    'cleaned_no_npwp': '5847834155340254',
    'cleaned_alamat': 'SODONG 5',
    'cleaned_NAMA_IBU_KANDUNG': 'DIFFERENT MOTHER',
    'CD_SP': '100103',
    'cleaned_name_cob': None,
    'cleaned_no_ktp_cob': None,
    'TGL_LAHIR_COBORR': None,
    'cleaned_alamat_cob': None
}
df_test = pd.DataFrame([sample_data])

In [25]:
# Connect to the database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",  
    database="skripsi" 
)

cursor = conn.cursor(dictionary=True)

# Fetch data from bigram_index
cursor.execute("SELECT * FROM bigram_index")

bigram_groups = cursor.fetchall()

bigram_dict = {
    row['bigram']: json.loads(row['group_values'])
    for row in bigram_groups
}

# Fetch data from bigram_index
cursor.execute("SELECT * FROM monre_dict")
monre_groups = cursor.fetchall()
monre_dict = {
    row['cleaned_name']: json.loads(row['place_index'])
    for row in monre_groups
}

In [26]:
def get_bigrams(name):
    # if not isinstance(name, str):  # Check if 'name' is a string
    #     print(f"Non-string value encountered: {name}")
    name = name.replace(" ", "")  # Remove spaces to consider all characters together
    return {name[i: i + 2] for i in range(len(name) - 1)}

# Function to compare two names based on bigrams and Jaro-Winkler similarity
def compare_bigrams_and_jaro_winkler(name1, name2, threshold=0.5):
    bigrams1 = get_bigrams(name1) # Use precomputed bigrams
    bigrams2 = get_bigrams(name2)  # Use precomputed bigrams
    
    # Find the common bigrams between the two names using set intersection
    common_bigrams = bigrams1.intersection(bigrams2)
    
    # Count how many common bigrams there are
    common_count = len(common_bigrams)
    
    # Find the smallest bigram length between the two names
    min_bigrams_len = min(len(bigrams1), len(bigrams2))
    
    # If common bigrams are greater than 50% of the smallest bigram set, calculate Jaro-Winkler
    if common_count > threshold * min_bigrams_len:
        jaro_winkler_similarity = distance.get_jaro_distance(name1, name2)
        return jaro_winkler_similarity
    else:
        return None

# Function to calculate Jaro-Winkler distances for names based on selected bigrams
def calculate_jaro_winkler_distances(name):
    bigrams = get_bigrams(name) # Use precomputed bigrams
    bigram_weights = [
        (bigram, len(bigram_dict[bigram]))
        for bigram in bigrams
        if bigram in bigram_dict
    ]
    
    # Sort bigrams by their frequency (lower frequency = higher weight)
    bigram_weights_sorted = sorted(bigram_weights, key=lambda x: x[1])

    # Select the top 3 least frequent bigrams
    selected_bigrams = [bg[0] for bg in bigram_weights_sorted[:5]]
    print(selected_bigrams)
    
    # Find all unique names in the groups of these 3 bigrams
    matching_names = set()
    for bigram in selected_bigrams:
        matching_names.update(bigram_dict[bigram])

    matching_names.discard(name)  # Remove the original name from comparison
    
    # Compute Jaro-Winkler similarity for each name in the matching group using bigram comparison
    distances = {}
    
    for other_name in matching_names:
        # Use the new comparison function to check bigram overlap and compute Jaro-Winkler
        similarity = compare_bigrams_and_jaro_winkler(name, other_name)
        if similarity and similarity > 0.75:
            distances[other_name] = similarity

    return name, distances  # Return the name and its distances

def add_new_name_to_results_dict(new_name):
    # Calculate Jaro-Winkler distances for the new name
    name, distances = calculate_jaro_winkler_distances(new_name)
    
    # If the new name has similar names, add it to results_list
    if distances:
        similar_names_df = pd.DataFrame(
            list(distances.items()), columns=["similar_name", "similarity"]
        )
        results_list = similar_names_df["similar_name"].tolist()  # Convert similar names to a list
    else:
        results_list = []
    results_list.append(new_name)
    return results_list

def jaro_winkler_match(value1, value2, threshold=0.92): # Default Threshold
    if pd.notna(value1) and pd.notna(value2):
        value1_str = str(value1).strip() 
        value2_str = str(value2).strip() 
        
        # Ensure neither value is an empty string
        if value1_str and value2_str:
            similarity = distance.get_jaro_distance(value1_str, value2_str)
            return similarity >= threshold
    return False 

In [35]:
for index, row in df_test.iterrows():
    if not monre_dict:
        kode_cabang = row['CD_SP']
        
        cursor.execute(f"SELECT COUNT FROM sp_count WHERE CD_SP = ({kode_cabang})")
        last_sequence = cursor.fetchone()
        
        sid_value = f"{kode_cabang}{(int(last_sequence['COUNT']) + 1):07d}"
        break
    
    row_compared = row
    matched_dfs = []
    results_list = add_new_name_to_results_dict(row['cleaned_name'])
    all_indices = set()
    for name in results_list:
        if name in monre_dict:
            indices = monre_dict[name]
            all_indices.update(map(int, monre_dict[name]))
            id_list = ','.join(map(str, all_indices))
            cursor.execute(f"SELECT * FROM credit_cust WHERE id IN ({id_list})")
            matched_dfs = cursor.fetchall()
            matched_dfs = pd.DataFrame(matched_dfs)
    
    if len(matched_dfs) == 0:
        kode_cabang = row['CD_SP']
        cursor.execute(f"SELECT COUNT FROM sp_count WHERE CD_SP = ({kode_cabang})")
        last_sequence = cursor.fetchone()
        
        sid_value = f"{kode_cabang}{(int(last_sequence['COUNT']) + 1):07d}"
        break
    else:
        result_df_nodup = matched_dfs.drop_duplicates(subset='NO_AGGR').reset_index(drop=True)
        
    name_compared = row['cleaned_name']
    dob_compared = row['TGL_LAHIR']
    tempat_compared = row['cleaned_TEMPAT_LAHIR']
    ktp_kitas_compared = row['cleaned_no_ktp']
    mother_name_compared = row['cleaned_NAMA_IBU_KANDUNG'] 
    npwp_compared = row['cleaned_no_npwp']
    address_compared = row['cleaned_alamat']
    
    print(result_df_nodup)

    # Filter result_df based on matching criteria
    filtered_result_df = result_df_nodup[
        ((pd.notna(result_df_nodup['TGL_LAHIR']) & pd.notna(dob_compared) & 
        (result_df_nodup['TGL_LAHIR'] == dob_compared)) |
        (pd.notna(result_df_nodup['cleaned_no_ktp']) & pd.notna(ktp_kitas_compared) & 
        (result_df_nodup['cleaned_no_ktp'] == ktp_kitas_compared)) |
        (pd.notna(result_df_nodup['cleaned_NAMA_IBU_KANDUNG']) & pd.notna(mother_name_compared) & 
        (result_df_nodup['cleaned_NAMA_IBU_KANDUNG'] == mother_name_compared)) |
        (pd.notna(result_df_nodup['cleaned_no_npwp']) & pd.notna(npwp_compared) & 
        (result_df_nodup['cleaned_no_npwp'] == npwp_compared))
    )]
    filtered_result_df = filtered_result_df.copy()
    filtered_result_df['flag_SID'] = 'N'  
    filtered_result_df['rule_num'] = None

    for index, row in filtered_result_df.iterrows():
        name_sim = jaro_winkler_match(row['cleaned_name'], name_compared)
        print( jaro_winkler_match(row['cleaned_name'], name_compared, threshold=0.95))
        print(pd.notna(row['cleaned_alamat']) and jaro_winkler_match(row['cleaned_alamat'], address_compared))
        print(pd.notna(row['TGL_LAHIR']) and row['TGL_LAHIR'] == dob_compared)
        print(pd.notna(row['cleaned_TEMPAT_LAHIR']) and row['cleaned_TEMPAT_LAHIR'] == tempat_compared)
        if (
            name_sim and 
            pd.notna(row['TGL_LAHIR']) and row['TGL_LAHIR'] == dob_compared and 
            pd.notna(row['cleaned_TEMPAT_LAHIR']) and row['cleaned_TEMPAT_LAHIR'] == tempat_compared and 
            pd.notna(row['cleaned_NAMA_IBU_KANDUNG']) and jaro_winkler_match(row['cleaned_NAMA_IBU_KANDUNG'], mother_name_compared)  # Rule 1
        ):
            filtered_result_df.loc[index, 'flag_SID'] = 'Y'
            filtered_result_df.loc[index, 'rule_num'] = 'RULE 1'

        elif (
            name_sim and 
            pd.notna(row['cleaned_no_ktp']) and row['cleaned_no_ktp'] == ktp_kitas_compared  # Rule 2
        ):
            filtered_result_df.loc[index, 'flag_SID'] = 'Y'
            filtered_result_df.loc[index, 'rule_num'] = 'RULE 2'

        elif (
            name_sim and 
            pd.notna(row['cleaned_no_npwp']) and row['cleaned_no_npwp'] == npwp_compared  # Rule 3
        ):
            filtered_result_df.loc[index, 'flag_SID'] = 'Y'
            filtered_result_df.loc[index, 'rule_num'] = 'RULE 3'

        elif (
            jaro_winkler_match(row['cleaned_name'], name_compared, threshold=0.95) and 
            pd.notna(row['cleaned_alamat']) and jaro_winkler_match(row['cleaned_alamat'], address_compared) and  # Rule 4
            pd.notna(row['cleaned_NAMA_IBU_KANDUNG']) and jaro_winkler_match(row['cleaned_NAMA_IBU_KANDUNG'], mother_name_compared)  
        ):
            filtered_result_df.loc[index, 'flag_SID'] = 'Y'
            filtered_result_df.loc[index, 'rule_num'] = 'RULE 4' 

        elif (
            pd.notna(row['TGL_LAHIR']) and row['TGL_LAHIR'] == dob_compared and 
            pd.notna(row['cleaned_no_ktp']) and row['cleaned_no_ktp'] == ktp_kitas_compared  # Rule 5
        ):
            filtered_result_df.loc[index, 'flag_SID'] = 'Y'
            filtered_result_df.loc[index, 'rule_num'] = 'RULE 5'  
        
        elif (
            jaro_winkler_match(row['cleaned_name'], name_compared, threshold=0.95) and 
            pd.notna(row['cleaned_alamat']) and jaro_winkler_match(row['cleaned_alamat'], address_compared) and
            pd.notna(row['TGL_LAHIR']) and row['TGL_LAHIR'] == dob_compared and 
            pd.notna(row['cleaned_TEMPAT_LAHIR']) and row['cleaned_TEMPAT_LAHIR'] == tempat_compared # Rule 6
        ):
            filtered_result_df.loc[index, 'flag_SID'] = 'Y'
            filtered_result_df.loc[index, 'rule_num'] = 'RULE 6'  

        else:
            filtered_result_df.loc[index, 'flag_SID'] = 'N'
            filtered_result_df.loc[index, 'rule_num'] = None # No match, no rule number
            
    filtered_result_df = filtered_result_df[filtered_result_df['flag_SID'] == 'Y'].reset_index(drop=True)
    
    if filtered_result_df.empty:
        row_compared_df = pd.DataFrame([row_compared])  
        row_compared_df['flag_SID'] = 'Y'                
        filtered_result_df = pd.concat([filtered_result_df, row_compared_df], ignore_index=True)

    check_sid_exist = False
    if not filtered_result_df['SID'].isnull().all():
        check_sid_exist = True
        sid_value = filtered_result_df['SID'].dropna().iloc[0]
        break

    # If no existing SID is found, generate a new one
    if not check_sid_exist:
        filtered_result_df = filtered_result_df.sort_values(by='DT_GOLIVE_VALID').reset_index(drop=True)
        
        kode_cabang = str(filtered_result_df.loc[0, 'CD_SP'])  
        
        cursor.execute(f"SELECT COUNT FROM sp_count WHERE CD_SP = ({kode_cabang})")
        last_sequence = cursor.fetchone()
        
        sid_value = f"{kode_cabang}{(int(last_sequence['COUNT']) + 1):07d}"
        filtered_result_df['SID'] = sid_value
          
print(sid_value)

['ON', 'IY', 'ED', 'DY', 'NO']
   id            NO_AGGR  NAME_GOLIVE flag_PC NAMA_IBU_KANDUNG SEX  \
0   1  12737446259973579  EDY PRIYONO       P          SUDARMI   M   

    TGL_LAHIR TEMPAT_LAHIR DT_GOLIVE_VALID   CD_SP  ... cleaned_TEMPAT_LAHIR  \
0  1967-10-01     SEMARANG      2025-06-06  600701  ...             SEMARANG   

  cleaned_name    cleaned_no_ktp cleaned_no_npwp cleaned_alamat  \
0  EDY PRIYONO  3374140110670015                         SODONG   

  cleaned_name_cob cleaned_no_ktp_cob cleaned_alamat_cob            SID  \
0                                                         6007010000001   

  SID_COBORR  
0       None  

[1 rows x 29 columns]
True
True
True
True
6007010000001


In [36]:
filtered_result_df['rule_num']

0    RULE 6
Name: rule_num, dtype: object

In [9]:
for index, row in df_test.iterrows():
    if not monre_dict:
        kode_cabang = row['CD_SP']
        
        cursor.execute(f"SELECT COUNT FROM sp_count WHERE CD_SP = ({kode_cabang})")
        last_sequence = cursor.fetchone()
        
        sid_value = f"{kode_cabang}{(int(last_sequence['COUNT']) + 1):07d}"
        
        break
    
    row_compared = row
    matched_dfs = []
    results_list = add_new_name_to_results_dict(row['cleaned_name'])
    all_indices = set()
    for name in results_list:
        if name in monre_dict:
            indices = monre_dict[name]
            all_indices.update(map(int, monre_dict[name]))
            id_list = ','.join(map(str, all_indices))
            cursor.execute(f"SELECT * FROM credit_cust WHERE id IN ({id_list})")
            matched_dfs = cursor.fetchall()
            matched_dfs = pd.DataFrame(matched_dfs)
    
    if len(matched_dfs) == 0:
        kode_cabang = row['CD_SP']
        
        cursor.execute(f"SELECT COUNT FROM sp_count WHERE CD_SP = ({kode_cabang})")
        last_sequence = cursor.fetchone()
        
        sid_value = f"{kode_cabang}{(int(last_sequence['COUNT']) + 1):07d}"
        
        break
    else:
        result_df_nodup = matched_dfs.drop_duplicates(subset='NO_AGGR').reset_index(drop=True)
    
        
    name_compared = row['cleaned_name']
    dob_compared = row['TGL_LAHIR']
    tempat_compared = row['cleaned_TEMPAT_LAHIR']
    ktp_kitas_compared = row['cleaned_no_ktp']
    mother_name_compared = row['cleaned_NAMA_IBU_KANDUNG'] 
    npwp_compared = row['cleaned_no_npwp']
    address_compared = row['cleaned_alamat']

    # Filter result_df based on matching criteria
    filtered_result_df = result_df_nodup[
        ((pd.notna(result_df_nodup['TGL_LAHIR']) & pd.notna(dob_compared) & 
        (result_df_nodup['TGL_LAHIR'] == dob_compared)) |
        (pd.notna(result_df_nodup['cleaned_no_ktp']) & pd.notna(ktp_kitas_compared) & 
        (result_df_nodup['cleaned_no_ktp'] == ktp_kitas_compared)) |
        (pd.notna(result_df_nodup['cleaned_NAMA_IBU_KANDUNG']) & pd.notna(mother_name_compared) & 
        (result_df_nodup['cleaned_NAMA_IBU_KANDUNG'] == mother_name_compared)) |
        (pd.notna(result_df_nodup['cleaned_no_npwp']) & pd.notna(npwp_compared) & 
        (result_df_nodup['cleaned_no_npwp'] == npwp_compared))
    )]

    filtered_result_df = filtered_result_df.copy()
    filtered_result_df['flag_SID'] = 'N'  
    filtered_result_df['rule_num'] = None
    for index, row in filtered_result_df.iterrows():
        name_sim = jaro_winkler_match(row['cleaned_name'], name_compared)
        
        if (
            name_sim and 
            pd.notna(row['TGL_LAHIR']) and row['TGL_LAHIR'] == dob_compared and 
            pd.notna(row['cleaned_TEMPAT_LAHIR']) and row['cleaned_TEMPAT_LAHIR'] == tempat_compared and 
            pd.notna(row['cleaned_NAMA_IBU_KANDUNG']) and jaro_winkler_match(row['cleaned_NAMA_IBU_KANDUNG'], mother_name_compared)  # Rule 1
        ):
            filtered_result_df.loc[index, 'flag_SID'] = 'Y'
            filtered_result_df.loc[index, 'rule_num'] = 'RULE 1'

        elif (
            name_sim and 
            pd.notna(row['cleaned_no_ktp']) and row['cleaned_no_ktp'] == ktp_kitas_compared  # Rule 2
        ):
            filtered_result_df.loc[index, 'flag_SID'] = 'Y'
            filtered_result_df.loc[index, 'rule_num'] = 'RULE 2'

        elif (
            name_sim and 
            pd.notna(row['cleaned_no_npwp']) and row['cleaned_no_npwp'] == npwp_compared  # Rule 3
        ):
            filtered_result_df.loc[index, 'flag_SID'] = 'Y'
            filtered_result_df.loc[index, 'rule_num'] = 'RULE 3'

        elif (
            jaro_winkler_match(row['cleaned_name'], name_compared, threshold=0.95) and 
            pd.notna(row['cleaned_alamat']) and jaro_winkler_match(row['cleaned_alamat'], address_compared) and  # Rule 4
            pd.notna(row['cleaned_NAMA_IBU_KANDUNG']) and jaro_winkler_match(row['cleaned_NAMA_IBU_KANDUNG'], mother_name_compared)  
        ):
            filtered_result_df.loc[index, 'flag_SID'] = 'Y'
            filtered_result_df.loc[index, 'rule_num'] = 'RULE 4' 

        elif (
            pd.notna(row['TGL_LAHIR']) and row['TGL_LAHIR'] == dob_compared and 
            pd.notna(row['cleaned_no_ktp']) and row['cleaned_no_ktp'] == ktp_kitas_compared  # Rule 5
        ):
            filtered_result_df.loc[index, 'flag_SID'] = 'Y'
            filtered_result_df.loc[index, 'rule_num'] = 'RULE 5'  
        
        elif (
            jaro_winkler_match(row['cleaned_name'], name_compared, threshold=0.95) and 
            pd.notna(row['cleaned_alamat']) and jaro_winkler_match(row['cleaned_alamat'], address_compared) and
            pd.notna(row['TGL_LAHIR']) and row['TGL_LAHIR'] == dob_compared and 
            pd.notna(row['cleaned_TEMPAT_LAHIR']) and row['cleaned_TEMPAT_LAHIR'] == tempat_compared # Rule 6
        ):
            filtered_result_df.loc[index, 'flag_SID'] = 'Y'
            filtered_result_df.loc[index, 'rule_num'] = 'RULE 6'  

        else:
            filtered_result_df.loc[index, 'flag_SID'] = 'N'
            filtered_result_df.loc[index, 'rule_num'] = None # No match, no rule number
    print(filtered_result_df)
    filtered_result_df = filtered_result_df[filtered_result_df['flag_SID'] == 'Y'].reset_index(drop=True)

    if filtered_result_df.empty:
        row_compared_df = pd.DataFrame([row_compared])  
        row_compared_df['flag_SID'] = 'Y'                
        filtered_result_df = pd.concat([filtered_result_df, row_compared_df], ignore_index=True)

    check_sid_exist = False
    if not filtered_result_df['SID'].isnull().all():
        check_sid_exist = True
        sid_value = filtered_result_df['SID'].dropna().iloc[0]
        break

    # If no existing SID is found, generate a new one
    if not check_sid_exist:
        filtered_result_df = filtered_result_df.sort_values(by='DT_GOLIVE_VALID').reset_index(drop=True)
        
        kode_cabang = str(filtered_result_df.loc[0, 'CD_SP']) 
        cursor.execute("SELECT COUNT FROM sp_count WHERE CD_SP = %s", (kode_cabang,))
        last_sequence = cursor.fetchone()
        
        sid_value = f"{kode_cabang}{(int(last_sequence['COUNT']) + 1):07d}"
        
        filtered_result_df['SID'] = sid_value
        
sid_value

   id            NO_AGGR  NAME_GOLIVE flag_PC NAMA_IBU_KANDUNG SEX  \
0   1  10822567375494397  EDY PRIYONO       P          SUDARMI   M   

    TGL_LAHIR TEMPAT_LAHIR DT_GOLIVE_VALID   CD_SP  ...    cleaned_no_ktp  \
0  1967-10-01     SEMARANG      2025-05-23  100103  ...  3374140110670015   

  cleaned_no_npwp cleaned_alamat cleaned_name_cob cleaned_no_ktp_cob  \
0                         SODONG                                       

  cleaned_alamat_cob            SID SID_COBORR flag_SID rule_num  
0                     1001030000001       None        N     None  

[1 rows x 31 columns]


'1001030000002'

In [13]:
distance.get_jaro_distance("EDI PRIYONO", "EDY PRIYONO")

0.92

In [10]:
name_compared

'EDI PRIYOTNO'

In [6]:
# SID COBORR
for index, row in df_test.iterrows():
    name_compared = row['cleaned_name_cob']
    name_borr_compared = row['cleaned_name']
    dob_compared = row['TGL_LAHIR_COBORR']
    ktp_kitas_compared = row['cleaned_no_ktp_cob']
    address_compared = row['cleaned_alamat_cob']
    sid_compared = sid_value

    if pd.notna(row['cleaned_name_cob']) and row['cleaned_name_cob'] != '' and row['cleaned_name_cob'] != 'nan':
        row_compared = row
        matched_dfs = []
        all_indices_coborr = set()
        id_list = None
        results_list = add_new_name_to_results_dict(row['cleaned_name_cob'])
        
        for name in results_list:
            if name in monre_dict:
                indices = monre_dict[name]
                all_indices_coborr.update(map(int, monre_dict[name]))
                id_list = ','.join(map(str, all_indices_coborr))
                cursor.execute(f"SELECT * FROM credit_cust WHERE id IN ({id_list})")
                rows_sql = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]
                matched_df = pd.DataFrame(rows_sql, columns=columns)
                matched_dfs.append(matched_df)

        if len(matched_dfs) > 0:
            combined_df = pd.concat(matched_dfs, ignore_index=True)
            mask = (
                (combined_df['TGL_LAHIR'] == dob_compared) |
                (combined_df['cleaned_no_ktp'] == ktp_kitas_compared)
            )
            filtered_result_df = combined_df[mask].copy()
            filtered_result_df['flag'] = None 
            
            for index, row in filtered_result_df.iterrows():
                name_sim = jaro_winkler_match(row['cleaned_name'], name_compared)
                if (
                    name_sim and 
                    pd.notna(row['cleaned_no_ktp']) and pd.notna(ktp_kitas_compared) 
                    and row['cleaned_no_ktp'] == ktp_kitas_compared  
                ):
                    filtered_result_df.loc[index, 'flag'] = 'Y'
                    filtered_result_df.loc[index, 'rule_num'] = 'RULE 1'
                elif (
                    pd.notna(row['TGL_LAHIR']) and pd.notna(dob_compared) and row['TGL_LAHIR'] == dob_compared and 
                    pd.notna(row['cleaned_no_ktp']) and pd.notna(ktp_kitas_compared) and row['cleaned_no_ktp'] == ktp_kitas_compared  # Rule 5
                ):
                    filtered_result_df.loc[index, 'flag'] = 'Y'
                    filtered_result_df.loc[index, 'rule_num'] = 'RULE 2'  
                else:
                    filtered_result_df.loc[index, 'flag'] = 'N'
                    filtered_result_df.loc[index, 'rule_num'] = None 

            filtered_result_df = filtered_result_df[filtered_result_df['flag'] == 'Y']
            if len(filtered_result_df) == 0 :
                print("1")
                sid_cobor_value = None
            else:
                if filtered_result_df['SID'].nunique() > 1:
                    print('Coborr dimiripkan dengan 2 SID berbeda yaitu:')
                    print(filtered_result_df['SID'].unique())
                else:
                    print("2")
                    sid_cobor_value = filtered_result_df['SID'].iloc[0]
        else:
            print("3")
            sid_cobor_value = None
    else:
        print("4")
        sid_cobor_value = None
print(sid_cobor_value)        

2
5005780000001


In [7]:
cursor.execute("SELECT * FROM mst_sid_borr_coborr")
mst_sid_borr_coborr = cursor.fetchall()
mst_sid_borr_coborr = pd.DataFrame(mst_sid_borr_coborr)

if len(mst_sid_borr_coborr)==0:
    mst_sid_borr_coborr = pd.DataFrame(columns=['SID_BORR','SID_COBORR','GID'])

In [11]:
exists = False

if pd.notna(sid_value) and pd.notna(sid_cobor_value):
    if not mst_sid_borr_coborr[
        (mst_sid_borr_coborr['SID_BORR'] == sid_value) &
        (mst_sid_borr_coborr['SID_COBORR'] == sid_cobor_value)
    ].empty:
        exists = True

elif pd.isna(sid_cobor_value):
    if not mst_sid_borr_coborr[
        mst_sid_borr_coborr['SID_BORR'] == sid_value
    ].empty:
        exists = True

In [14]:

added_to_mst_gid = False

# Append only if the pair does not exist
if not exists:
    new_row = pd.DataFrame([{
        'SID_BORR': sid_value,
        'SID_COBORR': sid_cobor_value,
        'GID': None
    }])
    mst_sid_borr_coborr = pd.concat([mst_sid_borr_coborr, new_row], ignore_index=True)
    added_to_mst_gid = True
    
if added_to_mst_gid:
    G = nx.Graph()
    for _, row in mst_sid_borr_coborr.iterrows():
        if pd.notna(row["SID_COBORR"]):  # Jika ada pasangan
            G.add_edge(row["SID_BORR"], row["SID_COBORR"])
        else:  # Jika tidak ada pasangan
            G.add_node(row["SID_BORR"])

    # Perbarui GID tanpa mengubah yang lama
    components = list(nx.connected_components(G))
    gid_mapping = {node: f"G{str(int(min(comp)))}" for comp in components for node in comp}
    mst_sid_borr_coborr["GID"] = mst_sid_borr_coborr["SID_BORR"].map(gid_mapping)
    print(mst_sid_borr_coborr)
     # Drop and recreate the SQL table
    cursor.execute("DROP TABLE IF EXISTS mst_sid_borr_coborr")
    cursor.execute("""
        CREATE TABLE mst_sid_borr_coborr (
            SID_BORR VARCHAR(13),
            SID_COBORR VARCHAR(13),
            GID VARCHAR(14)
        )
    """)
    conn.commit()

    # Insert data into SQL
    for _, row in mst_sid_borr_coborr.iterrows():
        cursor.execute("""
        INSERT INTO mst_sid_borr_coborr (SID_BORR, SID_COBORR, GID)
        VALUES (%s, %s, %s)
    """, (row["SID_BORR"], row["SID_COBORR"], row["GID"]))
    conn.commit()

        SID_BORR     SID_COBORR             GID
0  1001030000001           None  G1001030000001
1  1001940000001           None  G1001940000001
2  5005780000001           None  G1001940000001
3  1001940000001  5005780000001  G1001940000001


In [ ]:
if sid_cobor_value == None:
    gid_value = mst_sid_borr_coborr[mst_sid_borr_coborr['SID_BORR']==sid_value]['GID']
else:
    gid_value = mst_sid_borr_coborr[(mst_sid_borr_coborr['SID_BORR']==sid_value)&(mst_sid_borr_coborr['SID_COBORR']==sid_cobor_value)]['GID']
gid_value

0    G1001030000001
Name: GID, dtype: object